# Week 3: Feature Spaces en Collaborative Filtering

## Doel

Services als Bol.com, Amazon, YouTube, Netflix willen allemaal voorspellen welke producten (boeken, films, etc.) jij interesant vindt. In deze opdracht ga je zelf een systeem maken dat zulke voorspellingen kan doen. 

Vaak gebeurt dit door goed te kijken naar je gedrag op een website. Gebruikers bekijken producten, kopen producten en geven waarderingen (ratings). Deze informatie geeft een beeld van de producten die je leuk vindt. Vervolgens is de kunst om producten te vinden die daar op lijken. De kans is groot dat je die dan ook wel leuk vindt. 

Een manier om te achterhalen of bijvoorbeeld twee films op elkaar lijken is door de inhoud te vergelijken (gaan ze bijvoorbeeld over hetzelfde onderwerp). Dit heet _content based filtering_. Daar gaan we in de volgende opdracht naar kijken.

De opdracht deze week is gericht op _collaborative filtering_. Hierbij is het idee dat het collectieve gedrag van de gebruikers informatie geeft over de producten.

In collaborative filtering kan je weer twee strategieën onderscheiden: _user based filtering_ en _item based filtering_. Met user based filtering probeer je er achter te komen in welke mate gebruikers in gedrag op elkaar lijken en daarop gebaseerd een film aan te raden. Met item based filtering probeer je juist vast te stellen in welke mate twee films op elkaar lijken.

Welke aanpak is het best? Content based filtering? User based collaborative filtering? Item based collaborative filtering? Daar valt geen eenduidig antwoord op te geven. Dat blijkt enorm af te hangenen van de specifieke toepassing.

Het probleem wordt nog groter als je beseft dat bij elke aanpak weer een hoop subtiele keuzes te maken zijn die de resultaten beïnvloeden. Je kan bijvoorbeeld verschillende keuzes maken in een hoe je meet of twee producten (of gebruikers) op elkaar lijken. Hieronder kom je daar twee veelbebruikte maten voor tegen: _euclidean similarity_ en _cosine similarity_. En in de de praktijk zijn het er nog veel meer dan dat.

Omdat er geen eenduidig antwoord te geven is op de vraag: "Wat werkt het beste?" Is het enorm belangrijk om te weten hoe je de werking van het door jou samengestelde algoritme evalueert.  

## Opbouw

De opdracht van deze week bestaat uit twee delen en twee notebooks. In het eerste deel ga je verschillende varianten van een collaborative filtering systeem uitwerken. In het tweede deel ga je leren hoe je de kwaliteit van de voorspellingen van deze varianten kan evalueren.

## Studiemateriaal

- [Collaborative filtering, Harvard CS50](https://www.youtube.com/watch?v=Eeg1DEeWUjA)
- [Recommender systems handbook, Springerlink](https://link.springer.com/book/10.1007%2F978-0-387-85820-3) Hoofdstuk 1. Gratis te downloaden vanaf het UvA netwerk.

# Deel 1, collaborative filtering implementeren

Voor het maken van een collaborative filtering systeem maken we gebruik van de movieLens dataset die je ook in de vorige opdracht hebt gezien. We willen deze data gebruiken om gebruikers een film aan te raden die ze nog niet hebben gezien. Dit willen we doen gebaseerd op de ratings die de gebruiker voor _andere_ films heeft gegeven. 

We willen dus voor een gebruiker weten hoe goed hij of zij een bepaalde film _waarschijnlijk_ gaat vinden. Als we het probleem herformuleren in termen van ratings: Gegeven een gebruiker die een bepaalde film nog niet heeft gezien, _voorspel de rating_ die de gebruiker aan de film zou gegeven na het zien van de film.

## Aan de slag

We beginnen met het laden van de libraries die we nodig hebben:

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
np.seterr(divide='raise', over='raise', under='raise', invalid='raise')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D

# Item based filtering

Uiteindelijk ga je een film aanbevelen aan een gebruiker. Er zijn verschillende manieren om dat te doen. In deze opdracht ga je allereerst collaberative filtering toepassen. Het idee van collaberative filtering is dat we gebruik maken van de interactie van gebruikers om gebruikers items aan te raden. In de MovieLens dataset bestaat die interactie uit de ratings. 

Eerst kijken naar _item based filtering_. De intuitie is eenvoudig: als een gebruiker twee verschillende films dezelfde rating geeft, dan is dat een indicatie dat de films op elkaar lijken. Door alle ratings van alle gebruikers te beschouwen kunnen we een redelijk compleet beeld krijgen van de _similarity_ tussen twee films.

Dit deel bestaat uit de volgende stappen:
1. het lezen van de data
2. het transformeren van de data naar een bruikbaar formaat (utility matrix)
3. het bepalen van similarities
4. het doen van een aanbeveling

## 1.1: data lezen

We beginnen met een micro-dataset. Zo kan je straks makkelijk jouw code testen, want je hebt een overzicht over alle input. Deze kleine set bevat 3 films en 5 gebruikers. Dit is een subset van de [movielens](https://grouplens.org/datasets/movielens/) dataset. Ter vergelijking, de originele set bevat 27,000 films en 138,000 gebruikers. De algoritmes die we gaan toepassen werken in principe ook op de volledige dataset. Alhoewel we in deel drie zullen zien dat daar wel wat haken en ogen aan zitten.

De dataset bevat 3 tabellen (dataframes):
- **movies**, een tabel met de id van een film, de naam en het genre.
- **users**, een table met de id van de gebruiker en de naam.
- **ratings**, een tabel met de id van de user, de id van de film, de bijbehorende rating en de timestamp. 

We zullen in de onderstaande opdrachten alleen gebruik maken van de ratings tabel. De andere twee tabellen zijn er alleen voor onze eigen intuitie. Het is makkelijker praten over Shrek, dan film 4306.

Laad de data: 

In [ ]:
# Load the micro movielens set. This set contains 5 users and 3 movies
movies = pd.read_csv('./micro-movielens/movies.csv', index_col='movieId')
users = pd.read_csv('./micro-movielens/users.csv', index_col='userId')
ratings = pd.read_csv('./micro-movielens/ratings.csv')

display(movies)
display(users)
display(ratings)

### Vraag 1

In het algoritme dat we gaan maken hebben we vaak de rating voor een film en gebruiker nodig. Dus het is handig om een functie te maken die dit voor ons opzoekt. Hieronder staat de functie `get_rating()` gedefinieerd. Deze is nog niet helemaal af. Het is de bedoeling dat het de rating van een gebruiker voor een bepaalde film uit de tabel haalt. Bijvoorbeeld, `get_rating(ratings, 182, 2571)` zou `5.0` moeten geven. Randgeval: als er geen rating te vinden is voor de combinatie van film en gebruiker moet `get_rating()` de waarde `NaN` geven. Gebruik `np.nan` hiervoor. 

Implementeer `get_rating()` hieronder: 

In [ ]:
# define a helper function for accessing data
def get_rating(ratings, userId, movieId):
    """Given a userId and movieId, this function returns the corresponding rating.
       Should return NaN if no rating exists."""
    
    # TODO
    
    
# test: this should print '4.5'
rate = get_rating(ratings, 182, 2710)
print(f'The rating for user 182 and movie 2710 is {rate}')

### Check

In [ ]:
print('Check output type: ', end='')
_solution = get_rating(ratings, 182, 2710)
_expected_type = float
assert isinstance(_solution, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Check get_rating(): ', end='')
for _user, _movie, _solution in [(182, 2571, 5.0), (182, 2710, 4.5), (182, 4306, 4.0), (195, 2571, 3.0), (195, 2710, 1.0), (195, 4306, 3.0), (204, 2571, 4.5), (204, 2710, 5.0), (204, 4306, 4.0), (376, 2571, 3.5), (376, 2710, 1.5), (376, 4306, 4.0), (542, 2571, 5.0), (542, 2710, 0.5), (542, 4306, 5.0), (182, 1234, np.nan), (195, 5678, np.nan)]:
    if np.isnan(_solution):
        assert np.isnan(get_rating(ratings, _user, _movie)), f'Return NaN if there is no rating for the user/movie combination'
    else:
        assert get_rating(ratings, _user, _movie) == _solution, f'The rating for user {_user}, {_movie} should be {_solution}'
print('Success!')

## 1.2: feature spaces

Voor het analyseren van de data hebben we een tabel nodig waarin we makkelijk kunnen opzoeken wat de gebruikers van de films vonden. Zoals deze tabel:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>182</th><th>195</th><th>204</th><th>376</th><th>542</th></tr></thead><tbody><tr><th>2571</th><td>5.0</td><td>3.0</td><td>4.5</td><td>3.5</td><td>5.0</td></tr><tr><th>2710</th><td>4.5</td><td>1.0</td><td>5.0</td><td>1.5</td><td>0.5</td></tr><tr><th>4306</th><td>4.0</td><td>3.0</td><td>4.0</td><td>4.0</td><td>5.0</td></tr></tbody></table>

Zo'n tabel wordt de **utility matrix** genoemd. Hierin is het bijvoorbeeld makkelijk te zien dat gebruiker `542` (Nicolai) de film `2710` (The Blair Witch Project) knap waardeloos vond, met een rating van `0.5`.

We zijn bezig met item-based filtering, dus we willen films met elkaar kunnen vergelijken. Het is conventie om die dan op de verticale as te zetten. De rijen zijn dus de _items_ die we met elkaar willen vergelijken, en de kolommonen zijn de _eigenschappen_ van de items (in dit geval de ratings van de gebruikers). Deze eigenschappen worden meestal **features** genoemd. Elke film in deze tabel heeft dus vijf features: de rating van elke gebruiker.

### Vraag 2

De functie `pivot_ratings()` hieronder is nog niet af. Het is de bedoeling dat deze functie de bovenstaande tabel genereert. De functie maakt al een lege tabel aan met de juiste kolommen en rijen, het is aan jou om de juiste ratings in te vullen. Randgeval: als er geen rating te vinden is voor de combinatie van film en gebruiker moet er `NaN` op die plek in de tabel komen. Gebruik ook hier `np.nan`.

Tip: maak gebruik van de functie `get_rating()` die je in de bovenstaande opgaven hebt gedefinieerd.

In [ ]:
def pivot_ratings(ratings):
    """ takes a rating table as input and computes the utility matrix """
    # get movie and user id's
    movieIds = ratings['movieId'].unique()
    userIds = ratings['userId'].unique()
    
    # create empty data frame
    pivot_data = pd.DataFrame(np.nan, columns=userIds, index=movieIds, dtype=float)
    
    # use the function get_rating to fill the matrix
    # TODO
    
    return pivot_data

# test
utility_matrix = pivot_ratings(ratings)
display(utility_matrix)


### Check

In [ ]:
print('Check output type: ', end = '')
_student_answer = pivot_ratings(ratings)
_expected_type = pd.DataFrame
assert isinstance(_student_answer, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing pivot on complete data set: ', end = '')
_student_answer = pivot_ratings(ratings)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\x14@\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
pd.testing.assert_frame_equal(_student_answer, _solution, check_names=False, check_like=True)
print('success!')

print('Testing pivot on incomplete data set: ', end = '')
_incomplete_ratings = pd.read_msgpack(b"\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xa5Index\xa4name\xc0\xa5dtype\xa6object\xa4data\x94\xa6userId\xa7movieId\xa6rating\xa9timestamp\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7H\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x92\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x01\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xd7\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\xa5shape\x92\x01\t\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7\xd8\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\x8a\xa9\xde>\x00\x00\x00\x00\xffo`?\x00\x00\x00\x00f\xbc?;\x00\x00\x00\x00f6\x1bO\x00\x00\x00\x00!?\x1bO\x00\x00\x00\x00\xe8'\\Q\x00\x00\x00\x00\xf0)\\Q\x00\x00\x00\x00\xb0\xdfWE\x00\x00\x00\x00\x17\xe1WE\x00\x00\x00\x00\xa5shape\x92\x03\t\xa5dtype\xa5int64\xa5klass\xa8IntBlock\xa8compress\xc0")
_student_answer = pivot_ratings(_incomplete_ratings)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\x7f\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
pd.testing.assert_frame_equal(_student_answer, _solution, check_names=False, check_like=True)
print('success!')

De utility matrix geeft ons een hoop informatie over de relatie tussen films en de gebruikers. Dit kunnen we inzichtelijk maken door de data te visualiseren. Laten we eerst wat histograms maken:

In [ ]:
# create histograms for movie ratings
hist = utility_matrix.T.hist(bins=[0,1,2,3,4,5], layout=(1,5), figsize=(14,2));

# set axes
for ax in hist[0]:
    ax.set_ylim(0, 3.1)
    ax.set_xlim(-0.1, 5.1)

### Vraag 3

Gegeven de bovenstaande histograms wat is de _naam_ van de best beoordeelde films? 
En wat is de _naam_ van de film waarover de meningen het meest verdeeld zijn?

YOUR ANSWER HERE

De histogrammen geven nog niet zo heel veel informatie over de films. Door scatterplots te gebruiken kunnen we de data inzichtelijker maken.

In [ ]:
def plot_scatter(matrix, feature1, feature2, colors = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101']):
    # setup plot
    elements = matrix.shape[0]
    scatter = matrix.plot.scatter(feature1, feature2, color = colors[:elements])
    scatter.set_title('movie rating plot')
    
    # plot settings
    scatter.set_xlim(-0.1, 5.1)
    scatter.set_ylim(-0.1, 5.1)
    scatter.set_xlabel('rating for userId {}'.format(feature1))
    scatter.set_ylabel('rating for userId {}'.format(feature2))
    scatter.legend([mpatches.Circle((0,0), 1, fc=c) for c in colors], matrix.index, loc = 'lower right', title = 'movieId')
    
# scatter plot    
plot_scatter(utility_matrix, 542, 182)

In de bovenstaande plot staan de punten voor de drie films. De x-as geeft de rating van Ada (195) en de y-as geeft de rating Carl (376). We kunnen ons dus de films als punten in een abstracte ruimte voorstellen. Zo'n ruimte wordt een **feature space** genoemt.

### Vraag 4

Als je naar deze ruimte kijkt, welke films denk je dan dat het meest op elkaar lijken? Waarom?

YOUR ANSWER HERE

### Vraag 5

Plot de feature space zoals boven, maar dan voor Isaac en Gotlob.

Tip: Als je even snel wil kijken welke userId overeenkomt met welke naam kan je het commando `display(users)` gebruiken.

In [ ]:
# TODO


### Vraag 6

Als je naar _deze_ ruimte kijkt, welke films denk je _dan_ dat het meest op elkaar lijken?

YOUR ANSWER HERE

Door de ratings voor te stellen als een ruimte (een feature space) kan je kijken of twee films fysiek dicht bij elkaar zitten. En dat kan wat zeggen over of de films wellicht op elkaar lijken. Met andere woorden als veel gebruikers die The Matrix goed vonden ook een positieve beoordeling gegeven aan The Blair Witch Project, dan lijken die films wellicht op elkaar.

Een recommender system kan kiezen om iemand die The Matrix goed vond ook The Blair Witch Project aan te bevelen.

In de bovenstaande voorbeelden gebruikten we steeds een 2D-ruimte, een gebruiker per as. Zoals je zag kan de uitkomst flink verschillen per ruimte. Normaal gesproken gebruik je geen 2D-feature space, maar is het aantal dimensies gelijk aan het aantal features in de tabel. In ons voorbeeld zijn dat vijf dimensies (namelijk de ranking van de vijf gebruikers, Ada, Nicolai, Isaac, Gotlob en Carl). 

Helaas is een hoog-dimensionale feature space lastig te plotten. Met een beetje moeite kan je nog 3 dimensies plotten:

In [ ]:
def plot3d(matrix, feature1, feature2, feature3, colors = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101']):  
    # setup 3d plot
    fig = plt.figure()
    ax = Axes3D(fig)
    
    # enter data
    for i, ind in enumerate(matrix.index):
        scatter = ax.scatter(matrix.loc[ind, feature1], matrix.loc[ind, feature2], matrix.loc[ind, feature3], c = colors[i])

    # plot settings
    ax.legend([mpatches.Circle((0,0), 1, fc=c) for c in colors], matrix.index, loc = 'lower right', title = 'movieId')
    ax.set_xlabel('rating for userId {}'.format(feature1))
    ax.set_ylabel('rating for userId {}'.format(feature2))
    ax.set_zlabel('rating for userId {}'.format(feature3))
    
    plt.show()

plot3d(utility_matrix, 542, 182, 204)

Helaas is zo'n 3D-plot lastig te lezen. Welke films liggen er nou dichter bij elkaar in de bovenstaande plot? Bovendien, we hebben nog meer dimensies die we willen laten zien. Een gebruikelijke manier om zo'n hoger dimensionale ruimte te plotten is een _scatter plot matrix_. Hierin maken we alle scatterplots van alle combinaties van dimensies. Dit werkt al een stuk beter:

In [ ]:
def plot_scatter_matrix(matrix, colors = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101', '#000000', '#999999'], figsize = (8,8)):
    # setup plot
    max_val = matrix.max(axis = 1).max()
    min_val = matrix.min(axis = 1).min()
    fmatrix = matrix.fillna(min_val - 1)
    elements = fmatrix.shape[0]
    axarr = pd.plotting.scatter_matrix(fmatrix, s = 180.0, c = colors[:elements], figsize=figsize, alpha = 1.0, diagonal = '')
    m = fmatrix.shape[1]
    max_val = matrix.max(axis = 1).max()
    min_val = matrix.min(axis = 1).min()
    
    # set axes for all subplots
    for i in range(m):
        for j in range(m):
            axarr[i,j].set_xlim(min_val - 0.5, max_val + 0.5)
            axarr[i,j].set_ylim(min_val - 0.5, max_val + 0.5)

    # set labels for subplots
    labels = fmatrix.index 
    axarr[0,0].legend([mpatches.Circle((0,0),1,fc=c) for c in colors], labels, loc = 'lower left');

plot_scatter_matrix(utility_matrix)

### Vraag 7

a) Als je naar de bovenstaande scatterplot matrix kijkt, welke twee films denk je dat het meest op elkaar lijken? Beargumenteer.

b) Waarom zijn er geen plots getekend op de diagonaal?

YOUR ANSWER HERE

## 1.3: similarity

In het vorige deel hebben we gekeken of we aan de hand van de plots van de data konden zien welke films het meest op elkaar lijken. Nu gaan we kijken of we die intuities iets verder kunnen formaliseren. 

Uiteindelijk willen we een functie maken die, aan de hand van de gegeven data, kan uitreken in welke mate twee films op elkaar lijken.

Eerst moeten we een goede definitie van afstand hebben. De eenvoudigste manier is door te kijken naar de verschillen in de ratings van de gebruikers.

<table><thead><tr style="text-align: center;"><th></th><th>2571</th><th></th><th>4306</th><th></th><th>difference</th><th></th></tr></thead>
<tbody><tr><th>182</th><td>5.0</td><td>-</td><td>4.0</td><td>=</td><td>1.0</td><td>(Gotlob vindt The Matrix beter dan Shrek)</td></tr>
<tr><th>195</th><td>3.0</td><td>-</td><td>3.0</td><td>=</td><td>0.0</td><td>(Ada vindt beide films even goed)</td></tr>
<tr><th>204</th><td>4.5</td><td>-</td><td>4.0</td><td>=</td><td>0.5</td><td>(Isaac vindt The Matrix een klein beetje beter)</td></tr>
<tr><th>376</th><td>3.5</td><td>-</td><td>4.0</td><td>=</td><td>-0.5</td><td>(Carl vindt Shrek een klein beetje beter)</td></tr>
<tr><th>542</th><td>5.0</td><td>-</td><td>5.0</td><td>=</td><td>0.0</td><td>(Nicolai vindt beide films even goed)</td></tr>
</tbody></table>

Vervolgens berekenen we de afstand tussen de twee films door de som van de absolute waarde van deze verschillen te nemen:

$$
d = abs(1.0) + abs(0.0) + abs(0.5) + abs(-0.5) + abs(0.0) = 2.0
$$

Deze afstandsmaat noemen we de **manhattan distance** (kijk [hier](https://en.wikipedia.org/wiki/Taxicab_geometry) als je wil weten waarom dit zo heet). Let wel, dit is een afstandsmaat: de waarde wordt _groter_ naarmate de films _minder_ op elkaar lijken. In de praktijk willen we een gelijkheidsmaat: een waarde die groter wordt naarmate de films _meer_ op elkaar lijken. Een afstandsmaat is vrij eenvoudig om te zetten naar een gelijkheidsmaat. Er zijn meerdere manieren waarop dit kan, maar een veelgebruikte functie is:

$$
s = 1/(1 + d)
$$

Waarbij $s$ staat voor _similarity_ en $d$ voor de _distance_. In het geval van dit voorbeeld is de similarity dus:

$$
1 / (1 + 2.0) = 0.33
$$

Voor de volledigheid geven we ook de formele definitie van de manhattan distance. Geven item (film) $x$ en item $y$ met $n$ features (user ratings) defnieren we de manhattan distance $d(x, y)$ alsvolgt:

$$
d(x, y) = \Sigma_{i=1}^{n}|x_{i} - y_{i}|
$$ 

$$
s(x, y) = 1/(1 + d(x, y))
$$


Hier volgt de implementatie van de manhattan distance:

In [ ]:
def manhattan_distance(matrix, id1, id2):
    """Compute manhattan distance between two rows."""    
    # only take the features that have values for both id1 and id2
    selected_features = matrix.loc[id1].notna() & matrix.loc[id2].notna()
    
    # if no matching features, return NaN
    if not selected_features.any():
        return np.nan
    
    # get the features from the matrix
    features1 = matrix.loc[id1][selected_features]
    features2 = matrix.loc[id2][selected_features]
    
    # compute the distances for the features
    distances = features1 - features2
    
    # return the absolute sum
    return distances.abs().sum()

def manhattan_similarity(matrix, id1, id2):
    """Compute manhattan similarity between two rows."""
    # compute distance
    distance = manhattan_distance(matrix, id1, id2)
    
    # if no distance could be computed (no shared features) return a similarity of 0
    if distance is np.nan:
        return 0
    
    # else return similarity
    return 1 / (1 + distance)

similarity = manhattan_similarity(utility_matrix, 2571, 4306)
print(f'Similarity between The Matrix and Shrek: {similarity:.2f}')

Net als we eerder de utility matrix hadden gemaakt om gemakkelijk ratings op te kunnen zoeken, zo gaan we nu een matrix maken om makkelijk te kunnen zoeken naar similarity:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>2571</th><th>2710</th><th>4306</th></tr></thead><tbody><tr><th>2571</th><td>1.000000</td><td>0.095238</td><td>0.333333</td></tr><tr><th>2710</th><td>0.095238</td><td>1.000000</td><td>0.086957</td></tr><tr><th>4306</th><td>0.333333</td><td>0.086957</td><td>1.000000</td></tr></tbody></table>

### Vraag 8

De functie `create_similarity_matrix_manhattan()` hieronder is nog niet af. Er wordt al een matrix met de juiste dimensies gegenereerd, maar de rest is aan jou. Vul de matrix met de juiste similarities:

In [ ]:
def create_similarity_matrix_manhattan(matrix):
    """Creates the similarity matrix based on manhattan distance."""
    similarity_matrix = pd.DataFrame(0, index=matrix.index, columns=matrix.index, dtype=float)
    # TODO
    
    return similarity_matrix
           
similarity_matrix = create_similarity_matrix_manhattan(utility_matrix)
display(similarity_matrix)

### Check

In [ ]:
print('Check output type: ', end = '')
_student_answer = create_similarity_matrix_manhattan(utility_matrix)
_expected_type = pd.DataFrame
assert isinstance(_student_answer, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing create_similarity_matrix_manhattan on complete data set: ', end = '')
_student_answer = create_similarity_matrix_manhattan(utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\x18\x86a\x18\x86a\xb8?UUUUUU\xd5?\x18\x86a\x18\x86a\xb8?\x00\x00\x00\x00\x00\x00\xf0?d!\x0bY\xc8B\xb6?UUUUUU\xd5?d!\x0bY\xc8B\xb6?\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')

print('Testing create_similarity_matrix_manhattan on incomplete data set: ', end = '')
_utility_matrix = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\x7f\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
_student_answer = create_similarity_matrix_manhattan(_utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\x14;\xb1\x13;\xb1\xc3?\x00\x00\x00\x00\x00\x00\x00\x00\x14;\xb1\x13;\xb1\xc3?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')

### Vraag 9

In de bovenstaande matrix bevatten alle cellen op de diagonaal de waarde 1.0. Waarom is dat?

YOUR ANSWER HERE

Manhattan distance is een makkelijke maat om te gebruiken, maar wordt in de praktijk weinig gebruikt. Want stel, je hebt twee films met soortgelijke ratings, maar telkens met een klein verschil. Dan tel je bij de Manhattan distance alle verschillen in elke dimensie, hoe klein ze ook zijn, bij elkaar op. Daardoor lijkt het alsof de films toch veel verschillen.

Een populairdere maat is daarom de **Euclidische afstand**. Dit is de afstand die je krijgt als je de feature-space als een gewone fysieke ruimte beschouwd en de stelling van Pythagoras gebruikt om de afstand tussen twee punten (films) te bepalen. Door het gebruik van machten in de berekening van deze afstand worden kleine verschillen vrijwel genegeerd.

Net als bij de Manhattan distance kijken we eerst naar de verschillen in afstand.

<table><thead><tr style="text-align: center;"><th></th><th>2571</th><th></th><th>4306</th><th></th><th>difference</th><th></th></tr></thead>
<tbody><tr><th>182</th><td>5.0</td><td>-</td><td>4.0</td><td>=</td><td>1.0</td><td>(Gotlob vindt The Matrix beter dan Shrek)</td></tr>
<tr><th>195</th><td>3.0</td><td>-</td><td>3.0</td><td>=</td><td>0.0</td><td>(Ada vindt beide films even goed)</td></tr>
<tr><th>204</th><td>4.5</td><td>-</td><td>4.0</td><td>=</td><td>0.5</td><td>(Isaac vindt The Matrix een klein beetje beter)</td></tr>
<tr><th>376</th><td>3.5</td><td>-</td><td>4.0</td><td>=</td><td>-0.5</td><td>(Carl vindt Shrek een klein beetje beter)</td></tr>
<tr><th>542</th><td>5.0</td><td>-</td><td>5.0</td><td>=</td><td>0.0</td><td>(Nicolai vindt beide films even goed)</td></tr>
</tbody></table>

Vervolgens gebruiken we de stelling van Pythagoras voor het bereken van de euclidische afstand. In een 2D-ruimte is de stelling $d^2 = x^2 + y^2$ of, herschreven: $d = \sqrt{x^2 + y^2}$. In het 3D-geval is $d = \sqrt{x^2 + y^2 + z^2}$. In het algemeen is de stelling voor $n$ features $f_{1}$ t/m $f_{n}$:

$$
d = \sqrt{f_{1}^2 + f_{2}^2 + \ldots + f_{n}^2}
$$

In ons voorbeeld is de afstand $d$ dus:

$$
d = \sqrt{1.0^2 + 0.0^2 + 0.5^2 + -0.5^2 + 0.0^2} \approx 1.22
$$

Als we deze afstand omrekenen naar similarities:

$$
1 / (1 + 1.22) \approx 0.45
$$

De formele definitie:

$$
d(x, y) = \sqrt{\Sigma_{i=1}^{n}(x_{i} - y_{i})^2}
$$

$$
s(x, y) = 1/(1 + d(x, y))
$$

### Vraag 10

Implementeer hieronder de functie `create_similarity_matrix_euclid()`:

In [ ]:
# implement some helper functions?
# TODO


def create_similarity_matrix_euclid(matrix):
    """creates the similarity matrix based on eucledian distance"""
    # TODO
    

similarity = create_similarity_matrix_euclid(utility_matrix)
display(similarity)

### Check

In [ ]:
print('Check output type: ', end = '')
_student_answer = create_similarity_matrix_euclid(utility_matrix)
_expected_type = pd.DataFrame
assert isinstance(_student_answer, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing create_similarity_matrix_euclid on complete data set: ', end = '')
_student_answer = create_similarity_matrix_euclid(utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?tf\x9e\x0f\xa9\x1e\xc4?t\tI\xa0p\xc4\xdc?tf\x9e\x0f\xa9\x1e\xc4?\x00\x00\x00\x00\x00\x00\xf0?\x99-\xd1#\xdfJ\xc3?t\tI\xa0p\xc4\xdc?\x99-\xd1#\xdfJ\xc3?\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')

print('Testing create_similarity_matrix_euclid on incomplete data set: ', end = '')
_utility_matrix = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\x7f\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
_student_answer = create_similarity_matrix_euclid(_utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\xabq\xe5\x84\x99\n\xc7?\x00\x00\x00\x00\x00\x00\x00\x00\xabq\xe5\x84\x99\n\xc7?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')

Voor het vergelijken van verschillende items wordt in de praktijk veel van de _cosine similarity_ gebruik gemaakt. Voor het bepalen hiervan trekken we twee denkbeeldige lijnen van de oorsprong naar de data punten die we willen vergelijken en nemen we de hoek tussen die twee lijnen: 

<img src = 'cosine similarity.png'>

In het bovenstaande voorbeeld bepalen we de cosine similarity voor film 2571 en 2710 en kijken we even alleen naar de ratings van gebruiker 195 en 542. De hoek tussen de twee lijnen voor beide films $\Theta$ is $0.57^{\circ}$ rad. De cosine similarity is gedefinieerd als de cosinus van deze hoek. In dit voorbeeld is de similarity dus cos$(0.57) \approx 0.84$.

In het geval van de cosine similarity doet de afstand van een punt tot oorsprong er niet toe. Waarom zou dit een voordeel kunnen zijn?

Het bereken van de cosine similarity is eenvoudiger dan het voorbeeld doet vermoeden voor twee punten $A = (a_1, a_2)$ en $B = (b_1, b_2)$ is de cosine similarity:
$$
\textrm{cos}(A, B) = \frac{a_1*b_1 + a_2*b_2}{\sqrt{a_1^2 + a_2^2}*\sqrt{b_1^2 + b_2^2}}
$$

In ons geval is dat dus:
$$
\textrm{cos}(2571, 2710) = \frac{3.0*1.0 + 5.0*0.5}{\sqrt{3.0^2 + 5.0^2} * \sqrt{1.0^2 + 0.5^2}} \approx 0.84
$$

In de bovenstaande berekening hebben we voor het gemak aangenomen dat er maar twee features zijn (de ratings van gebruiker 195 en 542). In werkelijkheid hebben we natuurlijk niet twee features, maar veel meer. In ons geval hebben we er vijf. De algemene formule voor de cosinus van items met $n$ features $A = (a_1, a_2, \ldots, a_n)$ en $B = (b_1, b_2, \ldots, b_n)$ is:

$$
\textrm{cos}(A, B) = \frac{a_1*b_1 + a_2*b_2 + \ldots + a_n*b_n}{\sqrt{a_1^2 + a_2^2 + \ldots + a_n^2}*\sqrt{b_1^2 + b_2^2 + \ldots b_n^2}}
$$

In ons geval is dat dus:
$$
\textrm{cos}(2571, 2710) = \frac{5.0*4.5  + 3.0*1.0 + 4.5*5.0 + 3.5*1.5 + 5.0*0.5}{\sqrt{5.0^2 + 3.0^2 + 4.5^2 + 3.5^2 + 5.0^2} * \sqrt{4.5 ^ 2  + 1.0^2 + 5.0^2 + 1.5^2 + 0.5^2}} \approx 0.83
$$

De formele defnitie:
$$
\textrm{cos}(A, B) = \frac{\sum_{i = 1}^n a_i * b_i}{\sqrt{\sum_{i = 1}^n a_i^2}{\sqrt{\sum_{i = 1}^n b_i^2}}}
$$

### Vraag 11

Gebruik de bovenstaande formule voor het implementeren van de functie `create_similarity_matrix_cosine()` hieronder. Let erop dat als er geen overeenkomende features zijn, je `0.0` returned. Er is een randgeval bij een film die enkel wordt gerate met `0.0`, `return` ook dan `0.0`. Waarom is dit eigenlijk een randgeval bij het berekenen van cosine similarity?

In [ ]:
# TODO


def create_similarity_matrix_cosine(matrix):
    """ creates the similarity matrix based on cosine similarity """
    # TODO
    

similarity = create_similarity_matrix_cosine(utility_matrix)
display(similarity)

### Check

In [ ]:
print('Check output type: ', end='')
_student_answer = create_similarity_matrix_cosine(utility_matrix)
_expected_type = pd.DataFrame
assert isinstance(_student_answer, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing create_similarity_matrix_cosine on complete data set: ', end='')
_student_answer = create_similarity_matrix_cosine(utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\xff\xff\xff\xff\xff\xff\xef?\xbbSw\xb0\x1f\xb6\xea?\x17\xaf\x9f\x99`\xc5\xef?\xbbSw\xb0\x1f\xb6\xea?\x00\x00\x00\x00\x00\x00\xf0?2\x04d\xf4\x95\r\xe9?\x17\xaf\x9f\x99`\xc5\xef?2\x04d\xf4\x95\r\xe9?\xfe\xff\xff\xff\xff\xff\xef?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')

print('Testing create_similarity_matrix_cosine on incomplete data set: ', end='')
_utility_matrix = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\x7f\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
_student_answer = create_similarity_matrix_cosine(_utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\x9b+~]\xa2\xe2\xea?\x00\x00\x00\x00\x00\x00\x00\x00\x9b+~]\xa2\xe2\xea?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')

print("Testing create_similarity_matrix_cosine on a dataset with a movie that has only been rated with 0: ", end='')
_utility_matrix = DataFrame([[5.0, None, 4.5, None, 5.0], [4.5, None, 5.0, 1.5, 0.5], [0.0, 0.0, 0.0, 0.0, 0.0]], columns=[182,195,204,376,542], index=[2571, 2710, 4306])
_student_answer = create_similarity_matrix_cosine(_utility_matrix)
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\x9b+~]\xa2\xe2\xea?\x00\x00\x00\x00\x00\x00\x00\x00\x9b+~]\xa2\xe2\xea?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution, rtol=1e-1)
print('success!')



Het probleem met de cosine similarity is dat het erg gevoelig is voor bias. Dat wil zeggen, als de gemiddelde waarde van een feature geen 0 is, dan geeft de cosine similarity geen goede scores. In ons geval is dit een probleem. De gemiddelde rating per gebruiker is zeker geen 0.

De oplossing hiervoor is **mean centering**. Hierbij is het idee om de gemiddelde rating van een gebruiker te verminderen met het gemiddelde _van diezelfde_ gebruiker. Aangezien we in de utility matrix elke kolom de ratings van één gebruiker staan komt het er dus op neer elke kolom te verminderen met het gemiddelde van die kolom.

Hieronder zie je het effect hiervan. In Figuur 1 zien we de cosine similarity zonder mean centering, in Figuur 2 de cosine similairty van dezelfde twee punten, maar met centering. De cosine similarity toegepast op een mean centered utility matrix wordt ook wel de **adjusted cosine** genoemd. 

<table>
    <tr style ="background-color: #ffffff"><td style="text-align:center">Figuur 1</td><td style="text-align:center">Figuur 2</td></tr>
    <tr><td><img src = 'cosine similarity.png'></td><td><img src = 'adjusted cosine similarity.png'></td></tr>
</table>

### Vraag 12

Implementeer hieronder de functie `mean_center_columns()`:

In [ ]:
def mean_center_columns(matrix):
    # TODO
    


centered_utility_matrix = mean_center_columns(utility_matrix)
display(centered_utility_matrix)

similarity = create_similarity_matrix_cosine(centered_utility_matrix)
display(similarity)

### Check

In [ ]:
print('Check output type: ', end = '')
_student_answer = mean_center_columns(utility_matrix)
_expected_type = pd.DataFrame
assert isinstance(_student_answer, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing mean_center_columns on complete data set: ', end = '')
_student_answer = mean_center_columns(utility_matrix)
_solution1 = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0\xbfTUUUUU\xe5?VUUUUU\xf5\xbfTUUUUU\xe5?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xe0\xbf\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\xbf\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\x08\xc0\x00\x00\x00\x00\x00\x00\xf8?\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution1, rtol=1e-1)
print('success!')

print('Testing create_similarity_matrix_cosine on complete data set: ', end = '')
_student_answer = create_similarity_matrix_cosine(_solution1)
_solution2 = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\xd8\x9c\xe8p\xd0)\xee\xbfa\x96G\x1f\x97\xbd\xe9?\xd8\x9c\xe8p\xd0)\xee\xbf\x00\x00\x00\x00\x00\x00\xf0?\xa9\x15\x8f\x1ay\x9c\xee\xbfa\x96G\x1f\x97\xbd\xe9?\xa9\x15\x8f\x1ay\x9c\xee\xbf\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution2, rtol=1e-1)
print('success!')

print('Testing mean_center_columns on incomplete data set: ', end = '')
_utility_matrix = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\x7f\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
_student_answer = mean_center_columns(_utility_matrix)
_solution1 = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7(\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x05\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7(\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7x\x00\x00\x00\x00\x00\x00\x00\xd0?\x00\x00\x00\x00\x00\x00\xd0\xbf\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xd0\xbf\x00\x00\x00\x00\x00\x00\xd0?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x02@\x00\x00\x00\x00\x00\x00\x02\xc0\x00\x00\x00\x00\x00\x00\xf8\x7f\xa5shape\x92\x05\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution1, rtol=1e-1)
print('success!')

print('Testing create_similarity_matrix_cosine on incomplete data set: ', end = '')
_student_answer = create_similarity_matrix_cosine(_solution1)
_solution2 = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x03\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7H\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x03\x03\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(_student_answer, _solution2, rtol=1e-1)
print('success!')

## 1.4 Voorspellingen

We hebben nu meerdere maten voor het vergelijken van films. Hoe kunnen we de similarity functies gebruiken voor het aanbevelen van films. Dit probleem kunnen we herformuleren: Gegeven een gebruiker (de _target user_) die een bepaalde film (de _target film_) nog niet heeft gezien, hoe zou deze gebruiker de film raten na het zien?

De kern van het probleem is dus het voorspellen van een "missende" rating.

Er zijn veel mogelijke aanpakken, maar een veelgebruikte manier voor het aanbevelen van items is **neighborhood based**. De intuïtie is weer eenvoudig. Voor de _target film_, kijk welke films erop lijken (de _neighborhood_) en kijk wat de _target user_ daarvon vond. 

Het algoritme bestaat uit twee stappen:

1. Selecteer de _neighborhood_ van de _target film_. Selecteer alle films waarvoor geldt dat:
    - de _target user_ de film gezien heeft
    - de film zeer vergelijkbaar is met de _target film_
    
    
2. Neem het gewogen gemiddelde van de geselecteerde films. Dat wil zeggen dat een film die meer vergelijkbaar is met de _target film_ ook meer bijdraagt aan het gemiddelde.

Om deze stappen uit te werken hebben we iets meer data nodig. Voor het uitproberen van het algoritme voegen we handmatig wat data toe. We introduceren 2 extra gebruikers (123 en 456) en een extra film (4444):

In [ ]:
# create some mock data
l = [[123, 2571, 5.0, 0], [123, 2710, 1.5, 0], [123, 4444, 4.0, 0],
     [456, 2571, 1.5, 0], [456, 2710, 5.0, 0], [456, 4444, 1.0, 0],
     [182, 4444, 3.0, 0], [195, 4444, 2.0, 0], [204, 4444, 4.0, 0],
     [376, 4444, 4.5, 0], [542, 4444, 4.5, 0]]
extra_data = pd.DataFrame(l, columns = ratings.columns)
more_ratings = pd.concat([ratings, extra_data])

# recompute utility matrix
utility_matrix = pivot_ratings(more_ratings)
display(utility_matrix)

# recompute similarity matrix
centered_matrix = mean_center_columns(utility_matrix)
similarity = create_similarity_matrix_cosine(centered_matrix)
display(similarity)

Zoals je ziet hebben de nieuwe gebruikers (123 en 456) geen rating voor de film Shrek (4306). Die ratings gaan we dus proberen te voorspellen.

Om een beeld te krijgen van deze nieuwe data maken we weer een plot:

In [ ]:
plot_scatter_matrix(utility_matrix)

### Vraag 13

Als je naar de bovenstaande plot kijkt, welke film zou de film 4306 aanbevelen aan gebruiker 123? En aan gebruiker 456? Beargumenteer.

YOUR ANSWER HERE

### Vraag 14

We gaan eerst Stap 1 van het algoritme uitwerken:

1. Selecteer de neighborhood van de target film. Selecteer alle films waarvoor geldt dat:
    - de target user de film gezien heeft
    - de film zeer vergelijkbaar is met de target film

We moeten alleen een definitie geven van "zeer vergelijkbaar". Hiervoor kunnen we vele keuzes maken. Vaak worden de top $k$ meest vergelijkbare items genomen. Waarbij de beste waarde voor $k$ proefondervindelijk wordt vastgesteld. In het geval van deze, kleine dataset, is het handiger om gewoon alle items te nemen waarvan de _cosine similarity_ groter is dan 0.

Implementeer hieronder de functie `select_neighborhood()`. Deze functie geeft een `Series` terug met als index de `movieId`'s en als waardes de similarities. Voor bijvoorbeeld gebruker 123 en film 4306 geeft de functie de volgende `Series` terug:

    2571    0.732576
    4444    0.584364
    Name: 4306, dtype: float64
    
Let op: deze output bevat noch de film 2710 noch de film 4306. De film 2710 mist omdat de similarity kleiner is dan 0, en 4306 mist omdat gebruiker 123 deze nog niet heeft gezien.

In [ ]:
def select_neighborhood(similarity_matrix, utility_matrix, target_user, target_film):
    """selects all items with similarity > 0"""
    # TODO
    

neighborhood = select_neighborhood(similarity, utility_matrix, 123, 4306)
display(neighborhood)

### Check

In [ ]:
print('Check output type: ', end = '')
_solution = select_neighborhood(similarity, utility_matrix, 123, 4306)
_expected_type = pd.Series
assert isinstance(_solution, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Check select_neighborhood for user 123: ', end ='')
_student_answer = select_neighborhood(similarity, utility_matrix, 123, 4306)
_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\x83\xa3typ\xa9np_scalar\xa5dtype\xa5int64\xa4data\xa44306\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xd8\x00\x0b\n\x00\x00\x00\x00\x00\x00\\\x11\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xd8\x00\xe3\x93^WBq\xe7? \xf3\xfb\xd0\x1c\xb3\xe2?\xa8compress\xc0')
np.testing.assert_allclose(np.sort(_student_answer), np.sort(_solution), rtol=1e-1)
print('success!')

print('Check select_neighborhood for user 456: ', end ='')
_student_answer = select_neighborhood(similarity, utility_matrix, 456, 4306)
_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\x83\xa3typ\xa9np_scalar\xa5dtype\xa5int64\xa4data\xa44306\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xd8\x00\x0b\n\x00\x00\x00\x00\x00\x00\\\x11\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xd8\x00\xe3\x93^WBq\xe7? \xf3\xfb\xd0\x1c\xb3\xe2?\xa8compress\xc0')
np.testing.assert_allclose(np.sort(_student_answer), np.sort(_solution), rtol=1e-1)
print('success!')

### Vraag 15

Nu rest nog stap 2 van het algoritme:

2. Neem het gewogen gemiddelde van de geselecteerde films. Dat wil zeggen dat een film die meer vergelijkbaar is met de _target film_ ook meer bijdraagt aan het gemiddelde.

In het voorbeeld hierboven bevat de neighborhood de similarities $0.73$ en $0.58$ voor respectievelijk de films 2571 en 4444. De ratings die gebruiker 123 voor deze films heeft gegeven zijn $5.0$ en $4.0$. Het gewogen gemiddelde is dan:
$$
\textrm{predicted rating} = \frac{5.0 * 0.73+4.0 * 0.58}{0.73 + 0.58} \approx 4.56
$$

De formele definitie (met $r_i$ de rating en $s_i$ de similarity voor film $i$):
$$
\textrm{predicted rating} = \frac{\sum_{\textrm{i in neighborhood}} r_i * s_i}{\sum_{\textrm{i in neighborhood}}s_i}
$$

Implementeer hieronder de laatste stap van het item based collaborative filtering algoritme. Implementeer de functie `weighted_mean()`.

In [ ]:
def weighted_mean(neighborhood, utility_matrix, user_id):
    # TODO
    

    
neighborhood1 = select_neighborhood(similarity, utility_matrix, 123, 4306)
neighborhood2 = select_neighborhood(similarity, utility_matrix, 456, 4306)
prediction1 = weighted_mean(neighborhood1, utility_matrix, 123)
prediction2 = weighted_mean(neighborhood2, utility_matrix, 456)
# prediction2 = predict_rating_item_based(adjusted_cosine_similarity_matrix, eva, 4306)
print(f'User 123 predicted rating for Shrek {prediction1:.2f}')
print(f'User 456 predicted rating for Shrek {prediction2:.2f}')  

### Check

In [ ]:
print('Check output type: ', end = '')
_neighborhood1 = select_neighborhood(similarity, utility_matrix, 123, 4306)
_solution = weighted_mean(_neighborhood1, utility_matrix, 123)
_expected_type = float
assert isinstance(_solution, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print("Testing predictions: ", end='')
_neighborhood1 = select_neighborhood(similarity, utility_matrix, 123, 4306)
_prediction1 = weighted_mean(_neighborhood1, utility_matrix, 123)
assert _prediction1 > 4.0, "expected a high predicted rating for user 123"
assert _prediction1 > 0.0, "expected a predicted rating over 0.0"
assert _prediction1 < 5.0, "expected a predicted rating under 5.0"
_neighborhood2 = select_neighborhood(similarity, utility_matrix, 456, 4306)
_prediction2 = weighted_mean(_neighborhood2, utility_matrix, 456)
assert _prediction2 < 2.0,  "expected a high predicted rating for user 456"
assert _prediction2 > 0.0, "expected a predicted rating over 0.0"
assert _prediction2 < 5.0, "expected a predicted rating under 5.0"
print('success!')

### Vraag 16

Conclusie: Zou je gebruiker 123 Shrek aanraden? En gebruiker 456?

YOUR ANSWER HERE

# User based filtering

Tot nu toe keken we naar item based filtering. Hier voorspellen we ratings aan de hand van vergelijkbaarheid van de items (films). De voornaamste tegenhanger van item based filtering is *user based filtering*. Dit is het collaborative filtering equivalent van iemand met dezelfde filmsmaak om advies vragen. 

Het proces van user based filtering is vrijwel identiek aan item based filtering. We draaien alleen de rollen om:

- in plaats van een utility matrix met horizontaal de gebruikers en verticaal de films, wisselen we de assen om  
- in plaats van een similarity matrix voor films maken we een similarity matrix voor gebruikers
- in plaats van een neighborhood van de _target film_ selecteren we een neighborhood van de _target user_.
- in plaats van een gewogen gemiddelde van de ratings van vergelijkbare films nemen we een gewogen gemiddelde van de ratings van vergelijkbare users.

Voor de onderstaande vragen gaan we weer van de _uitgebreide ratings data_ uit.

Tip: Voor de onderstaande vragen hoef je maar weinig code te schrijven. **Maak zoveel mogelijk gebruik van de functies die je hierboven geschreven hebt.** Als je in een cel meer dan een paar regels code moet schrijven maak je het waarschijnlijk te ingewikkeld.

In [ ]:
display(more_ratings)

### Vraag 17

Maak een nieuwe utility matrix, maar nu met de gebruikers als rijen en de films als kolommen.

Tip: met `df.T` kan je de rijen en kolommen van dataframe `df` omwisselen.

In [ ]:
# make sure that utility_matrix contains the new flipped utility matrix

# TODO


display(utility_matrix)

### Check

In [ ]:
print('Check output type: ', end = '')
_expected_type = pd.DataFrame
assert isinstance(utility_matrix, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Check select_neighborhood for user 123: ', end ='')
_solution = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7 \x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\\\x11\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc78\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x00\x00\x00\x00\xc8\x01\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x04\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7\xe0\x00\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x0c@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x12@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x04\x07\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(utility_matrix, _solution, rtol=1e-1)
print('success!')

### Vraag 18

Maak een scatter plot matrix van de utility matrix.

In [ ]:
# TODO


### Vraag 19

Maak een mean centered utility matrix genaamd `centered_utility_matrix`. Let hierbij op: Dit is het enige waarbij de rollen niet zijn omgedraaid. We willen nog steeds de gemiddelde rating van de _gebruikers_ als offset gebruiken, niet die van de films. Het gaat er namelijk niet om hoe een gebruiker afwijkt van het gemiddelde rating van een film, maar wel hoe de gebruiker afwijkt van zijn gemiddelde rating bij een film. Met ander woorden, je moet hier de rijen centeren in plaats van de kolommen.

In [ ]:
# TODO


display(centered_utility_matrix)

### Check

In [ ]:
print('Check output type: ', end = '')
_expected_type = pd.DataFrame
assert isinstance(centered_utility_matrix, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing mean centered matrix on complete data set: ', end = '')
_solution1 = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7 \x00\x0b\n\x00\x00\x00\x00\x00\x00\x96\n\x00\x00\x00\x00\x00\x00\xd2\x10\x00\x00\x00\x00\x00\x00\\\x11\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc78\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x00\x00\x00\x00\xc8\x01\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x04\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc7 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc7\xe0\x00\x00\x00\x00\x00\x00\x00\xec?\x00\x00\x00\x00\x00\x00\xe8?\x00\x00\x00\x00\x00\x00\xc0?\x00\x00\x00\x00\x00\x00\xc0?\x00\x00\x00\x00\x00\x00\xf4?\x00\x00\x00\x00\x00\x00\xf8?\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\xd8?\x00\x00\x00\x00\x00\x00\xf4\xbf\x00\x00\x00\x00\x00\x00\xe4?\x00\x00\x00\x00\x00\x00\xfe\xbf\x00\x00\x00\x00\x00\x00\n\xc0\x00\x00\x00\x00\x00\x00\x00\xc0\x00\x00\x00\x00\x00\x00\x04@\x00\x00\x00\x00\x00\x00\xc0\xbf\x00\x00\x00\x00\x00\x00\xe8?\x00\x00\x00\x00\x00\x00\xd8\xbf\x00\x00\x00\x00\x00\x00\xe4?\x00\x00\x00\x00\x00\x00\xf4?\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf8\x7f\x00\x00\x00\x00\x00\x00\xf2\xbf\x00\x00\x00\x00\x00\x00\xd0\xbf\x00\x00\x00\x00\x00\x00\xd8\xbf\x00\x00\x00\x00\x00\x00\xf2?\x00\x00\x00\x00\x00\x00\xe8?\x00\x00\x00\x00\x00\x00\xe0?\x00\x00\x00\x00\x00\x00\xf8\xbf\xa5shape\x92\x04\x07\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(centered_utility_matrix, _solution1, rtol=1e-1)
print('success!')

### Vraag 20

Bereken de adjusted cosine similarity matrix voor de gebruikers en sla deze op in een variabele genaamd `similarity`.

In [ ]:
# TODO


### Check

In [ ]:
print('Check output type: ', end='')
_expected_type = pd.DataFrame
assert isinstance(similarity, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Testing similarity of complete data set: ', end = '')
_solution1 = pd.read_msgpack(b'\x84\xa3typ\xadblock_manager\xa5klass\xa9DataFrame\xa4axes\x92\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc78\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x00\x00\x00\x00\xc8\x01\x00\x00\x00\x00\x00\x00\xa8compress\xc0\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc78\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xc3\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x00\x00\x00\x00\xc8\x01\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6blocks\x91\x86\xa4locs\x86\xa3typ\xa7ndarray\xa5shape\x91\x07\xa4ndim\x01\xa5dtype\xa5int64\xa4data\xc78\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa6values\xc8\x01\x88\x00\x00\x00\x00\x00\x00\x00\xf0?\x89[\xf5&\t\x92\xc3?\x7fx4\xea\x893\xe5?!\xa2\x80\xce\xb5g\xe2\xbf\xb3h\x87\xfa\xb0\xca\xc9\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x07kr\xd6+\xa8\xd8?\x89[\xf5&\t\x92\xc3?\x00\x00\x00\x00\x00\x00\xf0?]t\xd1E\x17]\xe4\xbf\x9d>\xc9\x9fm=\xe6?\xe6\xe4\xab8\x9cd\xed?2\xd5)\x88\xbd\xb3\xed?\xcd\x91\xfc]\x96\x91\xe8\xbf\x7fx4\xea\x893\xe5?]t\xd1E\x17]\xe4\xbf\x00\x00\x00\x00\x00\x00\xf0?wV\xf1nZ\xb6\xee\xbf+5\xb3\xdaK\xd6\xea\xbf\x91*\xb0<>7\xe5\xbfX\x14\xfc\x8f\x19\x14\xec?!\xa2\x80\xce\xb5g\xe2\xbf\x9d>\xc9\x9fm=\xe6?wV\xf1nZ\xb6\xee\xbf\x00\x00\x00\x00\x00\x00\xf0?\xc3Y\xdaZTO\xed?\xc3\xfc\xe83\xdd\xc9\xe9?g\xc9\x8f\x96\xea\xcf\xee\xbf\xb3h\x87\xfa\xb0\xca\xc9\xbf\xe6\xe4\xab8\x9cd\xed?+5\xb3\xdaK\xd6\xea\xbf\xc3Y\xdaZTO\xed?\x00\x00\x00\x00\x00\x00\xf0?\xf4d\xc1\r4\xd5\xee?\xa5x\x9b\xb7\xd3\x9a\xee\xbf\x00\x00\x00\x00\x00\x00\x00\x002\xd5)\x88\xbd\xb3\xed?\x91*\xb0<>7\xe5\xbf\xc3\xfc\xe83\xdd\xc9\xe9?\xf4d\xc1\r4\xd5\xee?\x00\x00\x00\x00\x00\x00\xf0?]\xado\x80\x0b\x86\xed\xbf\x07kr\xd6+\xa8\xd8?\xcd\x91\xfc]\x96\x91\xe8\xbfX\x14\xfc\x8f\x19\x14\xec?g\xc9\x8f\x96\xea\xcf\xee\xbf\xa5x\x9b\xb7\xd3\x9a\xee\xbf]\xado\x80\x0b\x86\xed\xbf\x00\x00\x00\x00\x00\x00\xf0?\xa5shape\x92\x07\x07\xa5dtype\xa7float64\xa5klass\xaaFloatBlock\xa8compress\xc0')
np.testing.assert_allclose(similarity, _solution1, rtol=1e-1)
print('success!')

### Vraag 21

Bereken de neighborhood voor gebruiker 123 en 456 ten opzichte van film 4306. Sla de neighborhood van gebruiker 123 op in een variabele genaamd `neighborhood1` en voor gebruiker 456 in een variabele genaamd `neighborhood2`.

In [ ]:
# TODO


print(neighborhood1) # print the neighborhood for user 123
print(neighborhood2) # print the neighborhood for user 456

### Check

In [ ]:
print('Check output type: ', end = '')
_expected_type = pd.Series
assert isinstance(neighborhood1, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print('Check select_neighborhood for user 123: ', end='')
_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\x83\xa3typ\xa9np_scalar\xa5dtype\xa5int64\xa4data\xa3123\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xc7\x18\x00\xc3\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x00\x00\x00\x00\x1e\x02\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xc7\x18\x002\xd5)\x88\xbd\xb3\xed?\xc3\xfc\xe83\xdd\xc9\xe9?\xf4d\xc1\r4\xd5\xee?\xa8compress\xc0')
np.testing.assert_allclose(np.sort(neighborhood1), np.sort(_solution), rtol=1e-1)
print('success!')

print('Check select_neighborhood for user 456: ', end='')
_solution = pd.read_msgpack(b'\x87\xa3typ\xa6series\xa5klass\xa6Series\xa4name\x83\xa3typ\xa9np_scalar\xa5dtype\xa5int64\xa4data\xa3456\xa5index\x86\xa3typ\xa5index\xa5klass\xaaInt64Index\xa4name\xc0\xa5dtype\xa5int64\xa4data\xd8\x00\xb6\x00\x00\x00\x00\x00\x00\x00\xcc\x00\x00\x00\x00\x00\x00\x00\xa8compress\xc0\xa5dtype\xa7float64\xa4data\xd8\x00\x07kr\xd6+\xa8\xd8?X\x14\xfc\x8f\x19\x14\xec?\xa8compress\xc0')
np.testing.assert_allclose(np.sort(neighborhood2), np.sort(_solution), rtol=1e-1)
print('success!')

### Vraag 22

Bereken het gewogen gemiddelde voor de neighborhoods van user 123 en 456 (met de ratings voor film 4306).

In [ ]:
# TODO


print(f'User 123 predicted rating for Shrek {prediction1:.2f}')
print(f'User 456 predicted rating for Shrek {prediction2:.2f}') 

### Check

In [ ]:
print('Check output type: ', end = '')
_expected_type = float
assert isinstance(prediction1, _expected_type), f'expected output to be of type {_expected_type}, not {type(_solution)}'
print('success!')

print("Testing predictions: ", end='')
assert prediction1 > 3.0, "expected a predicted rating for user 123 above 3.0"
assert prediction1 < 5.0, "expected a predicted rating for user 123 under 5.0"
assert prediction2 > 3.0, "expected a predicted rating for user 456 above 3.0"
assert prediction2 < 5.0, "expected a predicted rating for user 456 under 5.0"
print('success!')

### Vraag 23

Als je deze user based voorspelling vergelijkt met de item based voorspelling, welke lijkt je dan beter? Beargumenteer in een paar korte zinnen. (In het volgende deel gaan we kijken hoe je de kwaliteit van de aanpak kan evalueren.)   

YOUR ANSWER HERE

# En nu?

Maar hoe kunnen we nu beoordelen hoe goed de algoritmes werken? Wat werk beter? Cosine similarity of euclidean similarity? User based of item based? In het volgende deel gaan we kijken hoe we de resultaten kunnen evalueren.